In [1]:
# Llibreries
import pandas as pd
import numpy as np
import re


In [2]:

# ===============================================
# Preprocés del fitxer "raw_olympic_athletes.csv"
# ===============================================

# Preprocés general
df_athletes = pd.read_csv("raw_olympic_athletes.csv", dtype={'athlete_year_birth':'Int64'})
df_athletes.drop(columns =['athlete_medals', 'bio'], inplace=True)

# Escriptura del dataframe en fitxer
df_athletes.to_csv("olympic_athletes.csv", index=False)


In [3]:
# ============================================
# Preprocés del fitxer "raw_olympic_hosts.csv"
# ============================================

# Preprocés general
df_hosts = pd.read_csv("raw_olympic_hosts.csv")
df_hosts.drop(columns =['game_end_date', 'game_start_date'], inplace=True)
df_hosts.rename(columns={"game_location": "game_country"}, inplace=True)
df_hosts["game_city"] = df_hosts["game_name"].str[:-5]

# Filtrar només per jocs d'estiu i a partir de l'any 1960 (dades del Banc Mundial)
df_hosts = df_hosts[df_hosts["game_season"] == "Summer"]
df_hosts = df_hosts[df_hosts["game_year"] >= 1960]

# Ordenar
df_hosts.sort_values(by="game_year", inplace=True)
df_hosts.reset_index(drop=True, inplace=True)

# Escriptura del dataframe en fitxer
df_hosts.to_csv("olympic_hosts.csv", index=False)

In [4]:
# =============================================
# Preprocés del fitxer "raw_olympic_medals.csv"
# =============================================

# Preprocés general
df_medals = pd.read_csv("raw_olympic_medals.csv")
df_medals.drop(columns =['participant_title', 'country_code'], inplace=True)
df_medals.rename(columns={"country_name": "IOC_country_name", "country_3_letter_code": "IOC_country_code"}, inplace=True)
df_medals["medal_type"] = df_medals["medal_type"].str.lower()

# Filtrar només per jocs d'estiu i a partir de l'any 1960 (dades del Banc Mundial)
slug_game_list = df_hosts["game_slug"].tolist()
df_medals = df_medals[df_medals["slug_game"].isin(slug_game_list)]
df_medals["game_year"] = df_medals["slug_game"].str[-4:].astype(int)

# Tractament de països: correció en base als codis i noms del Banc Mundial
df_country = pd.read_csv("country_conversion.csv")
df_medals = df_medals.merge(df_country[['IOC_country_code', 'WB_country_code', 'WB_country_name']], how="left", on='IOC_country_code')
df_medals.drop(columns =['IOC_country_name', 'IOC_country_code'], inplace=True)

# Determinar la ciutat organitzadora
df_medals = df_medals.merge(df_hosts[['game_year', 'game_city', 'game_name']], how="left", on='game_year')

# One-Hot Encoding de la columna "medal_type"
dummies = df_medals['medal_type'].str.get_dummies()
df_medals = pd.concat([df_medals, dummies], axis=1)

# Sumar les medalles (en aquest punt després del one-hot encoding la suma per fila sempre és 1)
df_medals["sum_medals"] = 1

# Escriptura del dataframe en fitxer
df_medals.to_csv("olympic_medals.csv", index=False)


In [5]:
# ====================================================
# Generació de fitxes derivats de "olympic_medals.csv"
# ====================================================

# ----------------------------------------------------
# "olympic_medals_year_country.csv"
# ----------------------------------------------------

# Agrupació per any i país, i suma de les medalles
df_medals_year_country = (
    df_medals.groupby(by=['game_year','WB_country_code', 'WB_country_name', 'game_city', 'game_name'])
    .agg({"gold": "sum", "silver": "sum", "bronze": "sum", "sum_medals": "sum"})
    .reset_index()
)

# Escriptura del dataframe en fitxer
df_medals_year_country.to_csv("olympic_medals_year_country.csv", index=False)

# ----------------------------------------------------
# "olympic_medals_country_year.csv"
# ----------------------------------------------------

# Agrupació per país i any, i suma de medalles
df_medals_country_year = (
    df_medals.groupby(by=['WB_country_code', 'WB_country_name', 'game_city', 'game_name', 'game_year'])
    .agg({"sum_medals": "sum"})
    .reset_index()
)

# Pivotar sobre país per transposar
df_medals_country_year = (
    df_medals_country_year.pivot(index=['WB_country_code', 'WB_country_name'], columns='game_year', values='sum_medals')
    .fillna(0)
    .astype(int)
    .rename_axis(None, axis=1)
    .reset_index() 
)

# Escriptura del dataframe en fitxer
df_medals_country_year.to_csv("olympic_medals_country_year.csv", index=False)

# ----------------------------------------------------
# "olympic_medals_country_year_cumsum.csv"
# ----------------------------------------------------

# Acumulat per país al llarg dels anys
df_medals_country_year_cumsum = df_medals_country_year.copy()
df_medals_country_year_cumsum.set_index(['WB_country_code', 'WB_country_name'], inplace=True)
df_medals_country_year_cumsum = df_medals_country_year_cumsum.cumsum(axis='columns')
df_medals_country_year_cumsum.reset_index(inplace=True)

# Afegir latitud i longitud del país
df_country_latitude_longitude = pd.read_csv("country_latitude_longitude.csv")
df_medals_country_year_cumsum = (
    df_medals_country_year_cumsum.merge(df_country_latitude_longitude[['WB_country_name', 'Latitude', 'Longitude']],
                                        how="left", on='WB_country_name')
)
df_medals_country_year_cumsum.set_index(['WB_country_code', 'WB_country_name', 'Latitude', 'Longitude'], inplace=True)
df_medals_country_year_cumsum.reset_index(inplace=True)
df_medals_country_year_cumsum_horizontal = df_medals_country_year_cumsum.copy()

# Pivotar amb "melt" per tenir el llistat per a tots els anys a nivell de fila
df_medals_country_year_cumsum = (
    df_medals_country_year_cumsum.melt(id_vars=["WB_country_code","WB_country_name","Latitude","Longitude"], 
                                       var_name="year", value_name="medals_cumsum")
)

# Escriptura del dataframe en fitxer
df_medals_country_year_cumsum.to_csv("olympic_medals_country_year_cumsum.csv", index=False)

# ----------------------------------------------------
# "olympic_medals_country_year_cumsum_horizontal.csv"
# ----------------------------------------------------

# Afegir continent i URL a la imatge de la bandera nacional
df_country_region_flag = pd.read_csv("country_region_flag.csv")
df_medals_country_year_cumsum_horizontal = (
    df_medals_country_year_cumsum_horizontal.merge(df_country_region_flag[['WB_country_name', 'Region', 'Image URL']],
                                        how="left", on='WB_country_name')
)
df_medals_country_year_cumsum_horizontal.set_index(['WB_country_code', 'WB_country_name', 'Latitude', 'Longitude', 'Region', 'Image URL'], inplace=True)
df_medals_country_year_cumsum_horizontal.reset_index(inplace=True)

# Escriptura del dataframe en fitxer
df_medals_country_year_cumsum_horizontal.to_csv("olympic_medals_country_year_cumsum_horizontal.csv", index=False)


In [6]:

# =========================================
# Preprocés del fitxer "raw_gdp_capita.csv"
# =========================================

# Preprocés general
df_gdp_capita = pd.read_csv("raw_gdp_capita.csv", skiprows=3, sep=';')
df_gdp_capita.drop(columns =['Indicator Name', 'Indicator Code', '2023'], inplace=True)
df_gdp_capita.rename(columns={"Country Name": "WB_country_name", "Country Code": "WB_country_code"}, inplace=True)

# Mitjana per any
df_gdp_capita_mean = df_gdp_capita.mean(numeric_only=True).to_frame(name="gdp_capita_average").transpose()

# Filtrar només els països que tenen medalla
medal_country_list = df_medals["WB_country_code"].unique().tolist()
df_gdp_capita = df_gdp_capita[df_gdp_capita["WB_country_code"].isin(medal_country_list)]

# Escriptura del dataframe en fitxer
df_gdp_capita.to_csv("gdp_capita.csv", index=False)

# ------------------------------------------------------------
# "gdp_capita_rank.csv" + "gdp_capita_rank_horizontal.csv"
# ------------------------------------------------------------

# Diferència sobre mitjana anual per país, calculada en % 
df_gdp_capita_rank = df_gdp_capita.copy()
for year in df_gdp_capita_mean.columns:
    average = df_gdp_capita_mean.loc["gdp_capita_average", year]
    df_gdp_capita_rank[year] = df_gdp_capita_rank[year].apply(lambda x: (x - average) / average * 100)

# Afegir latitud i longitud del país
df_gdp_capita_rank.drop(columns =['WB_country_name'], inplace=True)
df_gdp_capita_rank = (
    df_gdp_capita_rank.merge(df_country_latitude_longitude[['WB_country_code','WB_country_name', 'Latitude', 'Longitude']],
                             how="left", on='WB_country_code')
)
df_gdp_capita_rank.set_index(['WB_country_code', 'WB_country_name', 'Latitude', 'Longitude'], inplace=True)
df_gdp_capita_rank.reset_index(inplace=True)
df_gdp_capita_rank_horizontal = df_gdp_capita_rank.copy()

# Pivotar amb "melt" per tenir el llistat per a tots els anys a nivell de fila
df_gdp_capita_rank = (
    df_gdp_capita_rank.melt(id_vars=["WB_country_code","WB_country_name","Latitude","Longitude"],
                            var_name="year", value_name="GDP_deviation")
)

# Escriptura del dataframe en fitxer
df_gdp_capita_rank.to_csv("gdp_capita_rank.csv", index=False)
df_gdp_capita_rank_horizontal.to_csv("gdp_capita_rank_horizontal.csv", index=False)


In [7]:

# =========================================
# Preprocés del fitxer "raw_gdp.csv"
# =========================================

# Preprocés general
df_gdp = pd.read_csv("raw_gdp.csv", skiprows=3, sep=';')
df_gdp.drop(columns =['Indicator Name', 'Indicator Code', '2023'], inplace=True)
df_gdp.rename(columns={"Country Name": "WB_country_name", "Country Code": "WB_country_code"}, inplace=True)

# Filtrar només els països que tenen medalla
df_gdp = df_gdp[df_gdp["WB_country_code"].isin(medal_country_list)]

# Escriptura del dataframe en fitxer
df_gdp.to_csv("gdp.csv", index=False)

# ------------------------------------------------------------
# "gdp_melt.csv"
# ------------------------------------------------------------

df_gdp_melt= df_gdp.copy()
df_gdp_melt = df_gdp_melt.fillna(0)
df_gdp_melt.set_index(['WB_country_code', 'WB_country_name'], inplace=True)
#df_gdp_melt = df_gdp_melt.transform(lambda x: x / 1000000.0)
df_gdp_melt.reset_index(inplace=True)
df_gdp_melt = df_gdp_melt.melt(id_vars=["WB_country_code","WB_country_name"], var_name="year", value_name="GDP")

df_gdp_melt.to_csv("gdp_melt.csv", index=False)

# ------------------------------------------------------------
# "gdp_cumsum.csv"
# ------------------------------------------------------------

# Acumulat per país al llarg dels anys
df_gdp_cumsum = df_gdp.copy()
df_gdp_cumsum = df_gdp_cumsum.fillna(0)
df_gdp_cumsum.set_index(['WB_country_code', 'WB_country_name'], inplace=True)
df_gdp_cumsum = df_gdp_cumsum.cumsum(axis='columns')
#df_gdp_cumsum = df_gdp_cumsum.transform(lambda x: x / 1000000.0)
df_gdp_cumsum.reset_index(inplace=True)

# Pivotar amb "melt" per tenir el llistat per a tots els anys a nivell de fila
df_gdp_cumsum = (
    df_gdp_cumsum.melt(id_vars=["WB_country_code","WB_country_name"],
                            var_name="year", value_name="GDP_cumsum")
)

# Escriptura del dataframe en fitxer
df_gdp_cumsum.to_csv("gdp_cumsum.csv", index=False)


In [8]:
# ====================================================
# Generació de fitxes derivats dels fitxers anteriors
# ====================================================

# ------------------------------------------------------------
# "medals_gdp.csv"
# ------------------------------------------------------------

# Partim del dataset "df_medals_country_year_cumsum" i ho preparem per fer un merge
df_medals_gdp = df_medals_country_year_cumsum.copy()
df_medals_gdp.drop(columns =['Latitude', 'Longitude'], inplace=True)
df_medals_gdp["ctry_year"] = df_medals_gdp["WB_country_code"] + df_medals_gdp["year"].astype(str)

# Del dataset "df_gdp_capita_rank" ens quedem amb els anys olímpics 
year_list = df_medals_gdp['year'].unique().tolist()
year_list = list(map(str, year_list))
df_gdp_deviation = df_gdp_capita_rank.copy()
df_gdp_deviation.drop(columns =['Latitude', 'Longitude'], inplace=True)
df_gdp_deviation = df_gdp_deviation[df_gdp_deviation["year"].isin(year_list)]
df_gdp_deviation["ctry_year"] = df_gdp_deviation["WB_country_code"] + df_gdp_deviation["year"].astype(str)

# Fem merge entre els dos datasets per obtenir el % de desviació del GDP
df_medals_gdp = df_medals_gdp.merge(df_gdp_deviation[['ctry_year', 'GDP_deviation']], how="left", on='ctry_year')

# Afegim un camp que determina el signe de la desviació
df_medals_gdp["GDP_dev_sign"] = ["negative" if x < 0 else "positive" for x in df_medals_gdp["GDP_deviation"]]

# Del dataset "df_gdp_cumsum" ens quedem amb els anys olímpics
df_gdp_cumulative = df_gdp_cumsum.copy()
df_gdp_cumulative = df_gdp_cumulative[df_gdp_cumulative["year"].isin(year_list)]
df_gdp_cumulative["ctry_year"] = df_gdp_cumulative["WB_country_code"] + df_gdp_cumulative["year"].astype(str)

# Fem merge entre els dos datasets per obtenir l'acumulat del GDP
df_medals_gdp = df_medals_gdp.merge(df_gdp_cumulative[['ctry_year', 'GDP_cumsum']], how="left", on='ctry_year')

# Afegim el rati medalla-gdp
def ratio(row):
    if row['GDP_cumsum'] == 0:
        return None
    else:
        return row['medals_cumsum'] / row['GDP_cumsum']

df_medals_gdp["ratio_medal_gdp"] = df_medals_gdp.apply(ratio, axis=1)

# Afegim el continent
df_medals_gdp = df_medals_gdp.merge(df_country_region_flag[['WB_country_name', 'Region']], how="left", on='WB_country_name')

# Afegfim el GDP del país
df_gdp_melt_aux = df_gdp_melt.copy()
df_gdp_melt_aux = df_gdp_melt_aux[df_gdp_melt_aux["year"].isin(year_list)]
df_gdp_melt_aux["ctry_year"] = df_gdp_melt_aux["WB_country_code"] + df_gdp_melt_aux["year"].astype(str)
df_medals_gdp = df_medals_gdp.merge(df_gdp_melt_aux[['ctry_year', 'GDP']], how="left", on='ctry_year')

# Reindexem
df_medals_gdp.set_index(['WB_country_code', 'WB_country_name', 'Region', 'year'], inplace=True)
df_medals_gdp.reset_index(inplace=True)

# Eliminem la columna instrumental del merge
df_medals_gdp.drop(columns =['ctry_year'], inplace=True)

# Escriptura del dataframe en fitxer
df_medals_gdp.to_csv("medals_gdp.csv", index=False)
